In [1]:
################# User Inputs
# Paths
imDir = 'X:\\Force Project\\PublicationData\\SpinningDisk_Myosin\\3D_timelapse'
dataName = '20241107_U2OS_SGRLC_100Xoil_15mintimelapse_01_25plaser'
dataDir = imDir + '\\OpticalFlow3D_Python\\20241107_U2OS_SGRLC_100Xoil_15mintimelapse_01_25plaser'
saveDir = 'X:\\Force Project\\PublicationData\\SpinningDisk_Myosin\\PublicationFigures\\MovieS1'
# Metadata
xyscale = 0.065 # um/pixel
zscale = 0.2 # um/pixel
tscale = 45/60 # minutes/frame

################# Set up
# Notebook
import tifffile as tf
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import colorcet as cc

# Convert paths
imDir = Path(imDir)
dataDir = Path(dataDir)
saveDir = Path(saveDir)
# If the save folder doesn't exist, make it
saveDir.mkdir(exist_ok=True)

In [2]:
# Consistent visualization parameters
zSlice = 6 # ImageJ indexes from 1, so this matches the slice7 folder
ySlice = 1175-1-419 #  FIJI starts indexing from 1, and we have cropped, so this matches the 2D_yslice1175 folder

gap = 6 # How many vectors should be included? All vectors creates a messy plot
gapz = np.round(gap/2).astype(int) # Roughly account for anistropy in the visualization

# For clarity of visualization, the empty space around the cell is removed in these figures using these limits:
xa = 300
xb = 2075
ya = 420
yb = 1750

# Color Scale Limits
# Image intensity
intMin = 90
intMax = 1500
# Magnitude
cminMag = 0
cmaxMag = 0.25

In [ ]:
# Loop through to make the movie frames (which can be combined into any movie format)
for frame in np.arange(3,18):

    # Make images to match the quivers so that sizes, etc. are all consistent in cropping and so on
    imName = dataName + '.tif'
    allImages = tf.memmap(imDir / imName) # Does not load images into memory until called later

    imNow = allImages[frame,:]
    imNow = imNow[:,ya:yb,xa:xb]
        
    fig, ax = plt.subplots(2,2,height_ratios=[8, 1],sharex='col',layout="constrained")
    fig.set_figwidth(20)
    fig.set_figheight(10)

    ##### Original image, z
    ax[0,0].imshow(imNow[zSlice,:],cmap="gray",vmin=intMin,vmax=intMax) 
    ax[0,0].plot([20, 20+25/xyscale],[1300, 1300],'w',linewidth=8)

    tNow = frame*tscale
    mNow = np.floor(tNow)
    sNow = (tNow-mNow)*60
    tNow = str(int(mNow)).zfill(2) + ':' + str(int(sNow)).zfill(2)

    ax[0,0].annotate(tNow,[20,20],color='w',size=20)

    ax[0,0].axis('off')
    fig.set_facecolor("black")


    ##### Original image, y
    ax[1,0].imshow(imNow[:,ySlice,:],cmap="gray",vmin=intMin,vmax=intMax/2)

    plt.axis('off')
    fig.set_facecolor("black")
    ax[1,0].set_ylim([0,imNow.shape[0]])
    ax[1,0].set_aspect(zscale/xyscale)

    ##### Reliability thresholding
    relFile = dataName + '_rel_t' + str(frame).zfill(4) + '.tiff'
    rel = tf.imread(dataDir / relFile)
    relThresh = np.percentile(rel,90)
    relMask = rel > relThresh
    relMask = relMask[:,ya:yb,xa:xb] # Crop for clarity of visualization

    ####### Load and format velocity data
    vxFile = dataName + '_vx_t' + str(frame).zfill(4) + '.tiff'
    vx = tf.imread(dataDir / vxFile)
    vx = vx[:,ya:yb,xa:xb] # Crop for clarity of visualization
    vx = vx*relMask # Remove unreliable vectors
    vx[vx==0] = np.nan
    vx = vx*xyscale/tscale # Put into physical units

    vyFile = dataName + '_vy_t' + str(frame).zfill(4) + '.tiff'
    vy = tf.imread(dataDir / vyFile)
    vy = vy[:,ya:yb,xa:xb] # Crop for clarity of visualization
    vy = vy*relMask # Remove unreliable vectors
    vy[vy==0] = np.nan
    vy = vy*xyscale/tscale # Put into physical units

    vzFile = dataName + '_vz_t' + str(frame).zfill(4) + '.tiff'
    vz = tf.imread(dataDir / vzFile)
    vz = vz[:,ya:yb,xa:xb] # Crop for clarity of visualization
    vz = vz*relMask # Remove unreliable vectors
    vz[vz==0] = np.nan
    vz = vz*zscale/tscale # Put into physical units

    ##### Calculate useful quantities
    # Magnitude of intensity motion/change
    Magnitude = np.sqrt(np.power(vx,2)+np.power(vy,2)+np.power(vz,2))

    # Set up a grid for quiver plots
    x = np.linspace(0, vx.shape[2]-1, vx.shape[2])*xyscale
    y = np.linspace(0, vx.shape[1]-1, vx.shape[1])*xyscale
    z = np.linspace(0, vx.shape[0]-1, vx.shape[0])*zscale
    X, Y, Z = np.meshgrid(x,y,z)
    X = np.moveaxis(X,-1,0)
    Y = np.moveaxis(Y,-1,0)
    Z = np.moveaxis(Z,-1,0)

    ##### Quiver based on Magnitude: 2D view of a z-slice
    # Use a consistent colorscale across different figures
    norm = matplotlib.colors.Normalize(vmin=cminMag, vmax=cmaxMag)
    MagnitudePlot = norm(Magnitude[zSlice,::gap,::gap])
    MagnitudePlot = MagnitudePlot.flatten()
    colormap = cc.m_CET_L8

    # Make the quiver plot
    q = ax[0,1].quiver(X[zSlice,::gap,::gap],Y[zSlice,::gap,::gap],vx[zSlice,::gap,::gap],vy[zSlice,::gap,::gap],
            color=colormap(MagnitudePlot),
            scale=1/25,angles='xy',scale_units='xy',units='xy',headwidth=4)

    # Format the quiver plot
    # ax.set_facecolor("black")
    ax[0,1].set_xlim(np.min(X),np.max(X))
    ax[0,1].set_ylim(np.max(Y),np.min(Y))
    ax[0,1].set_aspect('equal')
    ax[0,1].axis('off')
    fig.set_facecolor("black")


    ###### Quiver based on Magnitude: 2D view of a y-slice
    # Use a consistent colorscale across different figures
    norm = matplotlib.colors.Normalize(vmin=cminMag, vmax=cmaxMag)
    MagnitudePlot = norm(Magnitude[::gapz,ySlice,::gap])
    MagnitudePlot = MagnitudePlot.flatten()
    colormap = cc.m_CET_L8

    # Make the quiver plot
    q = ax[1,1].quiver(X[::gapz,ySlice,::gap],Z[::gapz,ySlice,::gap],vx[::gapz,ySlice,::gap],vz[::gapz,ySlice,::gap],
            color=colormap(MagnitudePlot),
            scale=1/25,angles='xy',scale_units='xy',units='xy',headwidth=4)

    # Format the quiver plot
    # ax.set_facecolor("black")
    ax[1,1].set_xlim(np.min(X),np.max(X))
    ax[1,1].set_ylim(np.min(Z),np.max(Z))
    ax[1,1].set_aspect('equal')
    ax[1,1].axis('off')
    fig.set_facecolor("black")

    # Save and show
    saveName = 'MovieS1_Myosin3D_colorMagnitude_zSlice' + str(zSlice) + '_ySlice' + str(ySlice) + '_gap' + str(gap) + '_cmin' + str(cminMag) + '_cmax' + str(cmaxMag) + '_frame' + str(frame).zfill(4) + '.png'
    plt.savefig(saveDir / saveName, dpi=300, bbox_inches='tight')
    
    # plt.show()
    plt.close(fig)

